In [1]:
from collections import deque,defaultdict
from scipy.spatial import distance as dst
from matplotlib import collections as mc
import matplotlib.pyplot as plt
import shapely.geometry as sp
import distinctipy as ds
import pickle as pk
import pandas as pd
import numpy as np
import json
%matplotlib qt

<div>
<font size="4",style="font-family:Ubuntu"> <p> The following cell renders a matplotlib figure of the sector map (NodeMap) with the airports and saves the matplotlib figure as a pickle file which will be used as a background for rendering the flights when running the actual simulator on the website </p></font><br>
<font size="3">
<b>Input</b> &rarr; M_ConversionMetric.pkl, Airports.csv, ConvexDict.pkl, US_SectorMap.csv, airports.json <br>
<b>Output</b> &rarr; Simulator.pkl </font>
</div>
 

In [6]:
output = pk.load(open("Outputs/ConvexDict.pkl", "rb"))
df=pd.read_csv("Outputs/US_SectorMap.csv")
sectorNumber=0
dpi=10
fig=plt.figure(figsize=(1000,1000),dpi=dpi)
fig.tight_layout()
while sectorNumber<1250:
    x=output[sectorNumber]
    x1=list(map(lambda p:p[0],x))
    y1=list(map(lambda p:p[1],x))
    x2=list(df[df['sector']==sectorNumber]['easting'])
    y2=list(df[df['sector']==sectorNumber]['northing'])
    x3=sorted(x1)
    y3=sorted(y1)
    xmid=(x3[0]+x3[-1])/2
    ymid=(y3[0]+y3[-1])/2
    plt.text(xmid,ymid,f"{sectorNumber}",fontsize=55)
    plt.plot(x1,y1,'r')
    plt.plot([x1[0],x1[-1]],[y1[0],y1[-1]],'r')
    sectorNumber+=1
fig.tight_layout()
fig.canvas.draw()
pk.dump(fig,open("Outputs/Simulator.pkl","wb"))

<div>
<font size="4",style="font-family:Ubuntu"> <p> The following cell takes ad an input the ConnectedSectorGraph.pkl which houses the graph with the connecting points set as edge attributes, the CentroidDict.pkl and the Simulator.pkl generated in the previous cell. It renders a real time simulation of all the flight plans output by the GA from the file Sampleinput.txt with 2 minutes in-simulator time corresponding to 1 second real-time. </p></font><br>
<font size="3">
<b>Input</b> &rarr; ConnectedSectorGraph.pkl,CentroidDict.pkl,Simulator.pkl,SampleInput.txt <br>
<b>Output</b> &rarr; Simulator </font>
</div>

In [2]:
from scipy.spatial import distance as dst
from matplotlib import collections as mc
import matplotlib.pyplot as plt
import shapely.geometry as sp
import distinctipy as ds
import pickle as pk
import numpy as np
import pandas as pd
ConnectedSectorGraph = pk.load(open("Outputs/ConnectedSectorGraph.pkl", "rb"))
opfile=open("Outputs/SampleInput.txt","r")
fig = plt.figure(pk.load(open("Outputs/Simulator.pkl","rb")))
airportCoords = pk.load(open("Outputs/airportCoordDict.pkl","rb"))
ax = fig.axes[0]
SCALE_FACTOR=1
def path_maker(pathFromGA,MpMSpeed,index,Src,Dst):
    PointPath=[airportCoords[Src]]
    Distance=0.0
    for SectorIdx in range(len(pathFromGA)-1):
        Cur=pathFromGA[SectorIdx]
        Next=pathFromGA[SectorIdx+1]
        PointPath.append(ConnectedSectorGraph.es[ConnectedSectorGraph.get_eid(Cur,Next)]["ConnectingPoint"])
        Distance+=dst.euclidean(PointPath[-2],PointPath[-1])
    PointPath.append(airportCoords[Dst])
    Distance+=dst.euclidean(PointPath[-2],PointPath[-1])
    res=[]
    color=index
    DistanceDelta = MpMSpeed
    PathLine = sp.LineString(PointPath)
    distances = np.arange(0, Distance, DistanceDelta)
    points = [PathLine.interpolate(distance) for distance in distances] + [PathLine.boundary.geoms[1]]
    return points
lines=opfile.readlines()
TimeDict=[[] for i in range(1440)]
toStart=1440
toStop=0
DistinctColors=ds.get_colors(len(lines))
airports=set()
for index,line in enumerate(lines):
    if(line[-1]=='\n'):
        line = line[:-1]
    SplitLine=line.split(',')
    airports.add(SplitLine[-1])
    airports.add(SplitLine[-2])
    ActualStartTime=int(SplitLine[-6])
    path=[int(i) for i in SplitLine[:len(SplitLine)-8]]
    MpMSpeed=float(SplitLine[-3])*30.8667
    points=path_maker(path,MpMSpeed,index,SplitLine[-2],SplitLine[-1])
    toStop=max(toStop,len(points)+ActualStartTime)
    toStart=min(toStart,ActualStartTime)
    TimeDict[ActualStartTime].append([(points[0].x,points[0].y),(points[0].x,points[0].y),index])
    for ptIdx in range(1,len(points)):
        TimeDict[ActualStartTime+ptIdx].append([(points[ptIdx-1].x,points[ptIdx-1].y),(points[ptIdx].x,points[ptIdx].y),index])
ICAO=list(airports)
xAir=[]
yAir=[]
for air in ICAO:
    xAir.append(airportCoords[air][0])
    yAir.append(airportCoords[air][1])
plt.scatter(xAir,yAir,marker='o' , s=3000)
for i, txt in enumerate(ICAO):
    plt.annotate(txt, (xAir[i], yAir[i]),fontsize=70,color ="red")
Started=False
toQuit=False
UserInfo="Left-Click to start the Simulator \nRight-Click to Quit"
plt.text(1200000,500000,UserInfo,fontsize = 120)
toPlotNow=[]
CurTime=toStart
for i in range(toStop-toStart+1):
    c=[]
    mclc=[]
    for pointIndex in range(len(TimeDict[CurTime])):
        pointOne=TimeDict[CurTime][pointIndex][0]
        pointTwo=TimeDict[CurTime][pointIndex][1]
        color=TimeDict[CurTime][pointIndex][2]
        mclc.append([pointOne,pointTwo])
        c.append(DistinctColors[color])
    lc = mc.LineCollection(mclc, colors=c, linewidths=30)
    toPlotNow.append(lc)
    CurTime+=1
def StartSim(event):
    global CurTime
    global Started
    global toPlotNow
    if(not Started and event.button == 1):
        Started=True
        title="Hour Minutes";
        plt.text(6000000,3650000,title,fontsize = 120)
        CurTime=toStart
        plotIndex=0
        for CurTime in range(toStart,toStop+1,SCALE_FACTOR):
            if(toQuit):
                plt.close()
                break
            time=""
            if(CurTime//60<10):
                time+=f"0{CurTime//60}:"
            else:
                time+=f"{CurTime//60}:"
            if(CurTime%60<10):
                time+=f"0{CurTime%60}"
            else:
                time+=f"{CurTime%60}"
            time=plt.text(6000000, 3500000, time, fontsize = 220)
            for idx in range(plotIndex,min(plotIndex+SCALE_FACTOR,len(toPlotNow))):
                ax.add_collection(toPlotNow[idx])
            plotIndex+=SCALE_FACTOR
            fig.canvas.draw()
            time.remove()
            plt.pause(0.005)
def SimStopper(event):
    global toQuit
    if(event.button == 3):
        if(not Started):
            plt.close()
        toQuit=True
id1 = fig.canvas.mpl_connect('button_press_event', StartSim)
id2 = fig.canvas.mpl_connect('button_press_event', SimStopper)
plt.show()